In [1]:
#=============================================
# First part
#=============================================

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
!conda install -c conda-forge lxml --yes

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [5]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

In [6]:
htmldata = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [7]:
postalCodes=htmldata[0]

In [8]:
postalCodesFiltered=postalCodes[postalCodes.Borough != 'Not assigned']

In [9]:
postalCodesFiltered['Neighbourhood'].mask(postalCodesFiltered['Neighbourhood'] ==  'Not assigned', postalCodesFiltered.Borough, inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [10]:
postalCodesGrouped=postalCodesFiltered.groupby(['Postcode'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [11]:
postalCodesJoined = pd.merge(postalCodesGrouped, postalCodesFiltered[['Borough','Postcode']].drop_duplicates(), how='left', on=['Postcode', 'Postcode'])

In [12]:
postalCodesSorted=postalCodesJoined[['Postcode','Borough','Neighbourhood']].sort_values('Postcode', inplace=False)

In [13]:
 postalCodesSorted.shape

(103, 3)

In [14]:
#=============================================
# Second part
#=============================================

In [15]:
postalGeo_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [16]:
postalGeo_df.rename(columns={"Postal Code": "Postcode"},inplace =True)

In [17]:
canada_data_df = pd.merge(postalCodesSorted, postalGeo_df, how='left', on=['Postcode'])
canada_data_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
#=============================================
# Third part
#=============================================

In [19]:

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [20]:
borough='Toronto'

In [21]:
borough_toronto_df = canada_data_df.loc[canada_data_df['Borough'].str.contains(borough)]
borough_toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [22]:

borough_latitude = canada_data_df.loc[canada_data_df['Borough'].str.contains(borough), 'Latitude'].mean() # neighborhood latitude value
borough_longitude = canada_data_df.loc[canada_data_df['Borough'].str.contains(borough), 'Longitude'].mean() # neighborhood longitude value

In [23]:
borough_latitude

43.66727739999999

In [24]:
print("lat:{} lng:{}".format(borough_latitude,borough_longitude))

lat:43.66727739999999 lng:-79.39353346923077


In [25]:
# create map of Manhattan using latitude and longitude values
map_canada = folium.Map(location=[borough_latitude, borough_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(canada_data_df['Latitude'], canada_data_df['Longitude'], canada_data_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  

    
map_canada

In [26]:
CLIENT_ID = 'VFPQIHROZ2NZV5ODTHCDDPEEWTWD1RFT1ER4B3LACT5JIIP3' # your Foursquare ID
CLIENT_SECRET = 'KXUFQAIO4O2BN4W5I2A4CJISDTWNMFW2PSXP5BZ43MSTNXUD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VFPQIHROZ2NZV5ODTHCDDPEEWTWD1RFT1ER4B3LACT5JIIP3
CLIENT_SECRET:KXUFQAIO4O2BN4W5I2A4CJISDTWNMFW2PSXP5BZ43MSTNXUD


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
_venues = getNearbyVenues(names=borough_toronto_df['Neighbourhood'],
                                   latitudes=borough_toronto_df['Latitude'],
                                   longitudes=borough_toronto_df['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [29]:
_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


In [30]:
# one hot encoding
_onehot = pd.get_dummies(_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
_onehot['Neighbourhood'] = _venues['Neighbourhood'] 

# move Neighbourhood column to the first column
fixed_columns = [_onehot.columns[-1]] + list(_onehot.columns[:-1])
_onehot = _onehot[fixed_columns]

_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
_onehot.shape

(1681, 232)

In [32]:
_grouped = _onehot.groupby('Neighbourhood').mean().reset_index()
_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.011905,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.010638,0.00000,0.00000,0.00,0.031915,0.000000,0.042553,0.010638,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012048,0.0000,0.0000,0.000,0.0000,0.000,0.012048,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048


In [33]:
_grouped.shape

(38, 232)

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = _grouped['Neighbourhood']

for ind in np.arange(_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bar,Bakery,Salad Place,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Bakery,Steakhouse,Beer Bar,Farmers Market,Beach
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bakery,Nightclub,Pet Store,Climbing Gym,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Plane,Coffee Shop,Bar,Boat or Ferry,Sculpture Garden


In [36]:
#Run *k*-means to cluster the neighborhood into clusters.
# set number of clusters
kclusters = 3

_grouped_clustering = _grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [38]:
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bar,Bakery,Salad Place,Sushi Restaurant,Asian Restaurant
1,1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Bakery,Steakhouse,Beer Bar,Farmers Market,Beach
2,1,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bakery,Nightclub,Pet Store,Climbing Gym,Restaurant,Burrito Place
3,1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant
4,1,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Plane,Coffee Shop,Bar,Boat or Ferry,Sculpture Garden
5,1,"Cabbagetown, St. James Town",Coffee Shop,Pizza Place,Pub,Park,Italian Restaurant,Restaurant,Café,Bakery,Breakfast Spot,Jewelry Store
6,1,Central Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Café,Sandwich Place,Ice Cream Shop,Chinese Restaurant,Bubble Tea Shop,Bar,Spa
7,1,"Chinatown, Grange Park, Kensington Market",Café,Bar,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Record Shop
8,1,Christie,Grocery Store,Café,Park,Diner,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub
9,1,Church and Wellesley,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Pub,Bubble Tea Shop,Men's Store,Café


In [39]:
borough_toronto_df.shape

(39, 5)

In [40]:
neighborhoods_venues_indexed=neighborhoods_venues_sorted.set_index('Neighbourhood')

In [41]:
_merged=borough_toronto_df.reset_index(drop=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
_merged = _merged.join(neighborhoods_venues_indexed, on='Neighbourhood', how='inner')

In [42]:
_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Coffee Shop,Neighborhood,Trail,Pub,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Restaurant,Diner,Caribbean Restaurant,Pub
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Park,Pub,Food & Drink Shop,Burger Joint,Burrito Place,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Yoga Studio,Bakery,Italian Restaurant,American Restaurant,Latin American Restaurant,Fish Market,Bookstore,Neighborhood
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Lawyer,Swim School,Bus Line,Park,Gluten-free Restaurant,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [43]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[borough_latitude, borough_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


In [44]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(_merged['Latitude'], _merged['Longitude'], _merged['Neighbourhood'], _merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters